#### This notebook contains the classifier based on python pycaret module

In [ ]:
pip install pandas

In [ ]:
import pandas as pd

# Load the CSV file
file_path = 'access_logs.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataframe
df.head()


In [ ]:
pip install pycaret

In [ ]:
pip install --upgrade scipy scikitplot

In [ ]:
pip install scipy==1.8.0

In [ ]:
import pandas as pd
from pycaret.clustering import setup, create_model, evaluate_model, assign_model, save_model, plot_model

# Load the CSV file
file_path = 'access_logs2.csv'
df = pd.read_csv(file_path)

# Initialize the PyCaret clustering environment
clustering_setup = setup(data=df, session_id=123, normalize=True, verbose=False, memory=None)

# Create a specific clustering model, e.g., KMeans
kmeans_model = create_model('kmeans')

# Evaluate the model
evaluate_model(kmeans_model)

# Assign clusters to the data
clustered_data = assign_model(kmeans_model)

# Display the first few rows of the clustered data
print(clustered_data.head())

# Save the model
save_model(kmeans_model, 'clustering_model')

# Optional: Plot clusters
plot_model(kmeans_model, plot='cluster')


In [ ]:
import pandas as pd
from pycaret.clustering import setup, create_model, evaluate_model, assign_model, save_model, plot_model

# Load the CSV file
file_path = 'anomalous_csic2010_features.csv'
df = pd.read_csv(file_path)

# Initialize the PyCaret clustering environment
clustering_setup = setup(data=df, session_id=123, normalize=True, verbose=False, memory=None)

# Create a specific clustering model, e.g., KMeans
kmeans_model = create_model('kmeans')

# Evaluate the model
evaluate_model(kmeans_model)

# Assign clusters to the data
clustered_data = assign_model(kmeans_model)

# Display the first few rows of the clustered data
print(clustered_data.head())

# Save the model
save_model(kmeans_model, 'clustering_model')

# Optional: Plot clusters
plot_model(kmeans_model, plot='cluster')
